# 💡 Chapter A3: Lighting Systems (Phong Shading)

Textures alone won't make your game look realistic.

A flat-shaded sphere looks like a circle. A **lit** sphere looks 3D.

This chapter teaches the **Phong Lighting Model**, the foundation of all real-time rendering.

## 1. The Three Components of Light

Phong lighting has 3 parts:

### **Ambient** (Base Light)
- Simulates indirect light bouncing around the scene
- Prevents surfaces from being completely black
- Formula: `ambient = ambient_color * ambient_strength`

### **Diffuse** (Matte Reflection)
- Light scattered evenly in all directions
- Depends on angle between surface normal and light direction
- Formula: `diffuse = max(dot(normal, light_dir), 0.0) * light_color`

### **Specular** (Shiny Reflection)
- Mirror-like highlights
- Depends on viewing angle
- Formula: `specular = pow(max(dot(view_dir, reflect_dir), 0.0), shininess)`

## 2. Normal Vectors (The Key to Lighting)

A **normal** is a vector pointing **perpendicular** to a surface.

- Flat surfaces: all normals point the same way
- Curved surfaces: normals point in different directions

Normals are calculated per-vertex and interpolated across triangles.

In [ ]:
# Example: Normals for a cube
# Each face has its own normal
import numpy as np

# Front face normal (pointing toward +Z)
front_normal = [0.0, 0.0, 1.0]

# Top face normal (pointing toward +Y)
top_normal = [0.0, 1.0, 0.0]

# Right face normal (pointing toward +X)
right_normal = [1.0, 0.0, 0.0]

## 3. The Vertex Shader (Pass Normals to Fragment)

We need to transform normals correctly with the model matrix.

In [ ]:
vert_shader = '''
    #version 330
    in vec3 in_vert;
    in vec3 in_normal;
    in vec2 in_uv;
    
    uniform mat4 mvp;
    uniform mat4 model;  // For normal transformation
    
    out vec3 v_normal;
    out vec3 v_frag_pos;
    out vec2 v_uv;
    
    void main() {
        gl_Position = mvp * vec4(in_vert, 1.0);
        
        // Transform normal to world space
        v_normal = mat3(transpose(inverse(model))) * in_normal;
        
        // Fragment position in world space
        v_frag_pos = vec3(model * vec4(in_vert, 1.0));
        
        v_uv = in_uv;
    }
'''

## 4. The Fragment Shader (Calculate Lighting)

This is where the magic happens.

In [ ]:
frag_shader = '''
    #version 330
    in vec3 v_normal;
    in vec3 v_frag_pos;
    in vec2 v_uv;
    
    uniform vec3 light_pos;      // Position of the light
    uniform vec3 view_pos;       // Position of the camera
    uniform vec3 light_color;
    uniform sampler2D u_texture;
    
    out vec4 f_color;
    
    void main() {
        // 1. AMBIENT
        float ambient_strength = 0.1;
        vec3 ambient = ambient_strength * light_color;
        
        // 2. DIFFUSE
        vec3 norm = normalize(v_normal);
        vec3 light_dir = normalize(light_pos - v_frag_pos);
        float diff = max(dot(norm, light_dir), 0.0);
        vec3 diffuse = diff * light_color;
        
        // 3. SPECULAR
        float specular_strength = 0.5;
        vec3 view_dir = normalize(view_pos - v_frag_pos);
        vec3 reflect_dir = reflect(-light_dir, norm);
        float spec = pow(max(dot(view_dir, reflect_dir), 0.0), 32);
        vec3 specular = specular_strength * spec * light_color;
        
        // COMBINE
        vec3 result = (ambient + diffuse + specular) * texture(u_texture, v_uv).rgb;
        f_color = vec4(result, 1.0);
    }
'''

## 5. Passing Uniforms from Python

Send light and camera positions to the shader every frame.

In [ ]:
from pyrr import Vector3

# Light properties
light_pos = Vector3([2.0, 2.0, 2.0])
light_color = Vector3([1.0, 1.0, 1.0])  # White light

# In render loop:
prog['light_pos'].write(light_pos.astype('f4').tobytes())
prog['light_color'].write(light_color.astype('f4').tobytes())
prog['view_pos'].write(camera.position.astype('f4').tobytes())
prog['model'].write(model_matrix.astype('f4').tobytes())

## 6. Multiple Light Sources

Real scenes have many lights. We can't use loops in GLSL (performance), so we use **uniform arrays**.

In [ ]:
frag_shader_multi = '''
    #version 330
    
    struct Light {
        vec3 position;
        vec3 color;
        float intensity;
    };
    
    #define MAX_LIGHTS 4
    uniform Light lights[MAX_LIGHTS];
    uniform int num_lights;
    
    // ... (in variables)
    
    void main() {
        vec3 total_light = vec3(0.0);
        
        for (int i = 0; i < num_lights; i++) {
            // Calculate lighting for each light
            vec3 light_dir = normalize(lights[i].position - v_frag_pos);
            float diff = max(dot(v_normal, light_dir), 0.0);
            total_light += diff * lights[i].color * lights[i].intensity;
        }
        
        vec3 result = total_light * texture(u_texture, v_uv).rgb;
        f_color = vec4(result, 1.0);
    }
'''

## 7. Light Attenuation (Distance Falloff)

Real lights get dimmer with distance. Use the **inverse square law**.

In [ ]:
# In fragment shader:
'''
float distance = length(light_pos - v_frag_pos);
float attenuation = 1.0 / (1.0 + 0.09 * distance + 0.032 * distance * distance);
vec3 diffuse = diff * light_color * attenuation;
'''

## 🛠️ Challenge: The Lit Arena

Upgrade your multiplayer 3D arena with realistic lighting:

1. Implement Phong shading on all objects
2. Add 3 colored lights (red, blue, green) at different positions
3. Make the lights move or pulse over time
4. Add specular highlights to make surfaces look shiny

**Bonus:** Add a "spotlight" that follows the player (like a flashlight)

If your scene looks photorealistic instead of flat, you've mastered lighting. ✨